In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [21]:
# ========================================
# 0. CARGA DE DATOS
# ========================================

def cargar_datos_ciudades():
    """
    Carga el dataset de ciudades turísticas desde el archivo CSV
    """
    print("📂 CARGANDO DATASET DE CIUDADES TURÍSTICAS...")
    
    try:
        # Intentar cargar desde la carpeta data/raw
        df = pd.read_csv('../data/raw/Worldwide Travel Cities Dataset (Ratings and Climate).csv')
        print("✅ Dataset cargado exitosamente")
        
    except FileNotFoundError:
        try:
            # Intentar desde la carpeta actual
            df = pd.read_csv('Worldwide Travel Cities Dataset (Ratings and Climate).csv')
            print("✅ Dataset cargado exitosamente desde carpeta actual")
            
        except FileNotFoundError:
            print("⚠️ Archivo CSV no encontrado.")
                
    print(f"📊 Dataset cargado: {df.shape[0]} ciudades, {df.shape[1]} variables")
    return df

In [17]:
# ========================================
# 1. LIMPIEZA ESPECÍFICA PARA DATASET TURÍSTICO
# ========================================

def limpiar_datos_ciudades(df):
    """
    Realiza limpieza específica para el dataset de ciudades turísticas
    """
    print("="*70)
    print("🧹 LIMPIEZA DE DATOS - CIUDADES TURÍSTICAS")
    print("="*70)
    
    df_limpio = df.copy()
    
    print(f"📊 Dataset original: {df_limpio.shape[0]} ciudades, {df_limpio.shape[1]} variables")
     # 1.1 Tratamiento de valores nulos
    print("\n🔧 TRATAMIENTO DE VALORES NULOS:")
    
    nulos_antes = df_limpio.isnull().sum().sum()
    
    for columna in df_limpio.columns:
        nulos_col = df_limpio[columna].isnull().sum()
        
        if nulos_col > 0:
            print(f"   • {columna}: {nulos_col} valores nulos")
            
            # Estrategias específicas por tipo de variable
            if 'Rating' in columna:
                # Para ratings: usar mediana por continente si existe, sino mediana general
                if 'Continent' in df_limpio.columns:
                    df_limpio[columna] = df_limpio.groupby('Continent')[columna].transform(
                        lambda x: x.fillna(x.median())
                    )
                    # Si aún quedan nulos, usar mediana general
                    df_limpio[columna].fillna(df_limpio[columna].median(), inplace=True)
                else:
                    df_limpio[columna].fillna(df_limpio[columna].median(), inplace=True)
                print(f"     → Rellenado con mediana por grupo/general")
                
            elif columna in ['Average_Temperature', 'Annual_Rainfall', 'Humidity', 'Sunshine_Hours']:
                # Para datos climáticos: usar mediana por continente
                if 'Continent' in df_limpio.columns:
                    df_limpio[columna] = df_limpio.groupby('Continent')[columna].transform(
                        lambda x: x.fillna(x.median())
                    )
                    df_limpio[columna].fillna(df_limpio[columna].median(), inplace=True)
                else:
                    df_limpio[columna].fillna(df_limpio[columna].median(), inplace=True)
                print(f"     → Rellenado con mediana climática regional")
                
            elif df_limpio[columna].dtype in ['int64', 'float64']:
                # Otras variables numéricas: mediana
                mediana = df_limpio[columna].median()
                df_limpio[columna].fillna(mediana, inplace=True)
                print(f"     → Rellenado con mediana ({mediana:.2f})")
                
            else:
                # Variables categóricas: moda
                moda = df_limpio[columna].mode()
                if len(moda) > 0:
                    df_limpio[columna].fillna(moda[0], inplace=True)
                    print(f"     → Rellenado con moda ({moda[0]})")
    
    nulos_despues = df_limpio.isnull().sum().sum()
    print(f"\n✅ Valores nulos eliminados: {nulos_antes} → {nulos_despues}")
    
    # 1.2 Eliminar duplicados
    duplicados_antes = len(df_limpio)
    
    # Duplicados exactos
    df_limpio = df_limpio.drop_duplicates()
    duplicados_exactos = duplicados_antes - len(df_limpio)
    
    # Duplicados por ciudad (si existe la columna)
    if 'City' in df_limpio.columns:
        ciudades_duplicadas_antes = df_limpio['City'].duplicated().sum()
        if ciudades_duplicadas_antes > 0:
            print(f"\n⚠️ Encontradas {ciudades_duplicadas_antes} ciudades duplicadas:")
            duplicadas = df_limpio[df_limpio['City'].duplicated(keep=False)]['City'].value_counts()
            for ciudad, count in duplicadas.head().items():
                print(f"   • {ciudad}: aparece {count} veces")
            
            # Decidir estrategia: mantener la primera ocurrencia
            df_limpio = df_limpio.drop_duplicates(subset=['City'], keep='first')
            ciudades_eliminadas = ciudades_duplicadas_antes
            print(f"   → {ciudades_eliminadas} ciudades duplicadas eliminadas")
    
    print(f"\n🗑️ DUPLICADOS ELIMINADOS: {duplicados_exactos} filas exactas")
    
    # 1.3 Validar rangos de datos
    print(f"\n✅ VALIDACIÓN DE RANGOS:")
    
    # Validar ratings (deben estar entre 1-10)
    rating_columns = [col for col in df_limpio.columns if 'Rating' in col]
    for col in rating_columns:
        valores_fuera_rango = len(df_limpio[(df_limpio[col] < 1) | (df_limpio[col] > 10)])
        if valores_fuera_rango > 0:
            df_limpio[col] = np.clip(df_limpio[col], 1, 10)
            print(f"   • {col}: {valores_fuera_rango} valores ajustados al rango [1-10]")
        else:
            print(f"   • {col}: ✅ rango válido [1-10]")
    
    # Validar temperaturas (rango razonable -50 a +50°C)
    if 'Average_Temperature' in df_limpio.columns:
        temp_extremas = len(df_limpio[(df_limpio['Average_Temperature'] < -50) | 
                                     (df_limpio['Average_Temperature'] > 50)])
        if temp_extremas > 0:
            df_limpio['Average_Temperature'] = np.clip(df_limpio['Average_Temperature'], -50, 50)
            print(f"   • Average_Temperature: {temp_extremas} valores extremos ajustados")
    
    # Validar humedad (0-100%)
    if 'Humidity' in df_limpio.columns:
        humidity_fuera = len(df_limpio[(df_limpio['Humidity'] < 0) | (df_limpio['Humidity'] > 100)])
        if humidity_fuera > 0:
            df_limpio['Humidity'] = np.clip(df_limpio['Humidity'], 0, 100)
            print(f"   • Humidity: {humidity_fuera} valores ajustados al rango [0-100]")
    
    print(f"\n📊 Dataset limpio: {df_limpio.shape[0]} ciudades, {df_limpio.shape[1]} variables")
    return df_limpio


In [18]:
# ========================================
# 2. DETECCIÓN AVANZADA DE OUTLIERS
# ========================================

def detectar_outliers_turismo(df):
    """
    Detecta outliers específicos para datos turísticos
    """
    print("="*70)
    print("🎯 DETECCIÓN DE OUTLIERS - ANÁLISIS TURÍSTICO")
    print("="*70)
    
    outliers_info = {}
    columnas_numericas = df.select_dtypes(include=[np.number]).columns
    
    # Crear visualización de outliers
    n_cols = 2
    n_rows = int(np.ceil(len(columnas_numericas) / n_cols))
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
    fig.suptitle('🎯 Detección de Outliers por Variable', fontsize=16, fontweight='bold')
    
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, col in enumerate(columnas_numericas):
        row = i // n_cols
        col_idx = i % n_cols
        
        ax = axes[row, col_idx] if n_rows > 1 else axes[col_idx]
        
        # Boxplot para visualizar outliers
        bp = ax.boxplot(df[col].dropna(), patch_artist=True)
        bp['boxes'][0].set_facecolor('lightblue')
        bp['boxes'][0].set_alpha(0.7)
        
        # Calcular estadísticas de outliers
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        
        outliers_iqr = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]
        
        # Z-Score method
        z_scores = np.abs(stats.zscore(df[col].dropna()))
        outliers_zscore = len(z_scores[z_scores > 2])
        
        # Guardar información
        outliers_info[col] = {
            'iqr_count': len(outliers_iqr),
            'iqr_percentage': (len(outliers_iqr) / len(df)) * 100,
            'zscore_count': outliers_zscore,
            'limite_inf': limite_inferior,
            'limite_sup': limite_superior,
            'outliers_values': outliers_iqr[col].tolist() if len(outliers_iqr) < 10 else []
        }
        
        ax.set_title(f'{col}\nOutliers: {len(outliers_iqr)} ({(len(outliers_iqr)/len(df)*100):.1f}%)')
        ax.tick_params(axis='x', rotation=45)
        ax.grid(True, alpha=0.3)
    
    # Ocultar subplots vacíos
    for i in range(len(columnas_numericas), n_rows * n_cols):
        row = i // n_cols
        col_idx = i % n_cols
        ax = axes[row, col_idx] if n_rows > 1 else axes[col_idx]
        ax.set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Reportar outliers específicos para turismo
    print("\n📊 RESUMEN DE OUTLIERS POR VARIABLE:")
    for col, info in outliers_info.items():
        print(f"\n   🔍 {col}:")
        print(f"      • IQR Method: {info['iqr_count']} outliers ({info['iqr_percentage']:.1f}%)")
        print(f"      • Z-Score Method: {info['zscore_count']} outliers")
        print(f"      • Rango normal: [{info['limite_inf']:.2f}, {info['limite_sup']:.2f}]")
        
        if info['outliers_values']:
            print(f"      • Valores outliers: {info['outliers_values'][:5]}...")
    
    # Análisis especial para ciudades con ratings extremos
    if 'Overall_Rating' in df.columns and 'City' in df.columns:
        print(f"\n🌟 ANÁLISIS DE RATINGS EXTREMOS:")
        
        # Ciudades con ratings muy altos (posibles outliers positivos)
        high_rating_threshold = df['Overall_Rating'].quantile(0.95)
        ciudades_top = df[df['Overall_Rating'] >= high_rating_threshold]
        
        print(f"   📈 Ciudades con rating excepcional (>= {high_rating_threshold:.1f}):")
        for _, city in ciudades_top.head().iterrows():
            print(f"      • {city['City']}: {city['Overall_Rating']:.1f}/10")
        
        # Ciudades con ratings muy bajos (posibles outliers negativos)
        low_rating_threshold = df['Overall_Rating'].quantile(0.05)
        ciudades_bottom = df[df['Overall_Rating'] <= low_rating_threshold]
        
        print(f"   📉 Ciudades con rating bajo (<= {low_rating_threshold:.1f}):")
        for _, city in ciudades_bottom.head().iterrows():
            print(f"      • {city['City']}: {city['Overall_Rating']:.1f}/10")
    
    return outliers_info


In [19]:
# ========================================
# 3. CREACIÓN DE NUEVAS VARIABLES
# ========================================

def crear_variables_derivadas_turismo(df):
    """
    Crea nuevas variables específicas para análisis turístico
    """
    print("="*70)
    print("🔄 CREACIÓN DE VARIABLES DERIVADAS")
    print("="*70)
    
    df_transformado = df.copy()
    variables_creadas = 0
    
    # 3.1 Categorías de rating general
    if 'Overall_Rating' in df_transformado.columns:
        df_transformado['Rating_Category'] = pd.cut(
            df_transformado['Overall_Rating'],
            bins=[0, 5, 6.5, 8, 10],
            labels=['Bajo', 'Medio', 'Alto', 'Excelente'],
            include_lowest=True
        )
        variables_creadas += 1
        print("   ✅ 'Rating_Category' creada (Bajo/Medio/Alto/Excelente)")
    
    # 3.2 Índice climático (combinación de temperatura y lluvia)
    if 'Average_Temperature' in df_transformado.columns and 'Annual_Rainfall' in df_transformado.columns:
        # Normalizar variables para el índice
        temp_norm = (df_transformado['Average_Temperature'] - df_transformado['Average_Temperature'].min()) / \
                   (df_transformado['Average_Temperature'].max() - df_transformado['Average_Temperature'].min())
        
        # Invertir lluvia (menos lluvia = mejor clima)
        rain_norm = 1 - ((df_transformado['Annual_Rainfall'] - df_transformado['Annual_Rainfall'].min()) / \
                        (df_transformado['Annual_Rainfall'].max() - df_transformado['Annual_Rainfall'].min()))
        
        df_transformado['Climate_Index'] = (temp_norm * 0.6 + rain_norm * 0.4) * 10
        variables_creadas += 1
        print("   ✅ 'Climate_Index' creada (índice combinado clima)")
    
    # 3.3 Categorías de costo
    if 'Cost_Index' in df_transformado.columns:
        df_transformado['Cost_Category'] = pd.cut(
            df_transformado['Cost_Index'],
            bins=[0, 50, 75, 100, float('inf')],
            labels=['Económico', 'Moderado', 'Caro', 'Premium']
        )
        variables_creadas += 1
        print("   ✅ 'Cost_Category' creada (Económico/Moderado/Caro/Premium)")
    
    # 3.4 Índice turístico compuesto (si tenemos múltiples ratings)
    rating_columns = [col for col in df_transformado.columns if 'Rating' in col and col != 'Overall_Rating']
    
    if len(rating_columns) >= 3:
        # Crear promedio ponderado de ratings específicos
        weights = {
            'Attractions_Rating': 0.25,
            'Culture_Rating': 0.20,
            'Food_Rating': 0.20,
            'Safety_Rating': 0.15,
            'Shopping_Rating': 0.10,
            'Nightlife_Rating': 0.10
        }
        
        tourism_index = 0
        weight_sum = 0
        
        for col in rating_columns:
            if col in weights:
                tourism_index += df_transformado[col] * weights[col]
                weight_sum += weights[col]
            else:
                tourism_index += df_transformado[col] * 0.05  # Peso por defecto
                weight_sum += 0.05
        
        df_transformado['Tourism_Index'] = tourism_index / weight_sum if weight_sum > 0 else tourism_index
        variables_creadas += 1
        print("   ✅ 'Tourism_Index' creada (índice turístico compuesto)")
    
    # 3.5 Clasificación por volumen turístico
    if 'Tourist_Volume' in df_transformado.columns:
        # Usar percentiles para clasificar
        p25 = df_transformado['Tourist_Volume'].quantile(0.25)
        p50 = df_transformado['Tourist_Volume'].quantile(0.50)
        p75 = df_transformado['Tourist_Volume'].quantile(0.75)
        
        df_transformado['Tourism_Volume_Category'] = pd.cut(
            df_transformado['Tourist_Volume'],
            bins=[0, p25, p50, p75, float('inf')],
            labels=['Baja', 'Moderada', 'Alta', 'Masiva']
        )
        variables_creadas += 1
        print("   ✅ 'Tourism_Volume_Category' creada")
    
    # 3.6 Índice de estacionalidad (si tenemos best visit month)
    if 'Best_Visit_Month' in df_transformado.columns:
        # Mapear meses a estaciones (hemisferio norte por defecto)
        season_map = {
            12: 'Invierno', 1: 'Invierno', 2: 'Invierno',
            3: 'Primavera', 4: 'Primavera', 5: 'Primavera',
            6: 'Verano', 7: 'Verano', 8: 'Verano',
            9: 'Otoño', 10: 'Otoño', 11: 'Otoño'
        }
        
        df_transformado['Best_Season'] = df_transformado['Best_Visit_Month'].map(season_map)
        variables_creadas += 1
        print("   ✅ 'Best_Season' creada (estación óptima de visita)")
    
    # 3.7 Ratio precio-calidad
    if 'Cost_Index' in df_transformado.columns and 'Overall_Rating' in df_transformado.columns:
        # Ratio: mayor valor indica mejor relación calidad-precio
        df_transformado['Value_for_Money'] = df_transformado['Overall_Rating'] / (df_transformado['Cost_Index'] / 100)
        variables_creadas += 1
        print("   ✅ 'Value_for_Money' creada (relación calidad-precio)")
    
    print(f"\n📊 RESUMEN DE TRANSFORMACIÓN:")
    print(f"   • Variables originales: {len(df.columns)}")
    print(f"   • Variables nuevas creadas: {variables_creadas}")
    print(f"   • Variables totales: {len(df_transformado.columns)}")
    
    return df_transformado

In [20]:
# ========================================
# 4. FUNCIÓN PRINCIPAL FASE 2
# ========================================

def main_fase2_ciudades(df):
    """
    Ejecuta toda la fase 2 de limpieza para dataset turístico
    """
    print("🚀 INICIANDO EDA - FASE 2: LIMPIEZA Y PREPROCESAMIENTO")
    print("Dataset: Worldwide Travel Cities Ratings and Climate")
    print("="*70)
    
    # Guardar copia original para comparación
    df_original = df.copy()
    
    # 1. Limpiar datos
    print("🔧 PASO 1: LIMPIEZA DE DATOS")
    df_limpio = limpiar_datos_ciudades(df)
    
    # 2. Detectar outliers
    print("\n🎯 PASO 2: DETECCIÓN DE OUTLIERS")
    outliers_info = detectar_outliers_turismo(df_limpio)
    
    # 3. Crear variables derivadas
    print("\n🔄 PASO 3: CREACIÓN DE VARIABLES DERIVADAS")
    df_final = crear_variables_derivadas_turismo(df_limpio)
    
    # 4. Resumen final
    print("\n" + "="*70)
    print("✅ FASE 2 COMPLETADA EXITOSAMENTE")
    print(f"📊 Dataset procesado: {df_final.shape[0]} ciudades, {df_final.shape[1]} variables")
    
    # Mostrar cambios realizados
    print(f"\n📈 CAMBIOS REALIZADOS:")
    print(f"   • Filas: {len(df_original)} → {len(df_final)}")
    print(f"   • Columnas: {len(df_original.columns)} → {len(df_final.columns)}")
    print(f"   • Valores nulos eliminados: {df_original.isnull().sum().sum()} → {df_final.isnull().sum().sum()}")
    
    # Guardar dataset limpio
    try:
        df_final.to_csv('data/processed/cities_cleaned_and_transformed.csv', index=False)
        print(f"   • Dataset guardado: data/processed/cities_cleaned_and_transformed.csv")
    except:
        print(f"   • No se pudo guardar el dataset (directorio no existe)")
    
    print("📝 Próximo paso: Análisis exploratorio detallado")
    
    return df_final, outliers_info

# Ejemplo de uso
if __name__ == "__main__":
    # Aquí cargarías tu df desde la fase 1
    # df_final, outliers_info = main_fase2_ciudades(df)
    pass